In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import *
from ibapi.order import *
import ibapi

import requests
import bs4 as bs
import datetime
import time
import threading

from sklearn.linear_model import Ridge
import talib as ta
from talib import MA_Type

In [2]:
init_event = threading.Event() # init event
id_event = threading.Event()
mkt_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [3]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        self.mkt_price = []
        # create a connection with IBKR
        self.connect(address, port, cid)
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.total_value = 0
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib: ibapi.common.TickAttrib):
            print("Tick Price. Ticker Id:", reqId, "tickType:", tickType, "Price:", price)
            if tickType == 9: # if tickType is Close Price
                self.mkt_price.append([reqId, price])
                mkt_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.concat([self.last_portfolio,
                                        pd.DataFrame([[contract.symbol, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL]],
                                                    columns=self.last_portfolio.columns)],
                                        ignore_index=True)
        port_event.set()

    # query total value
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        self.total_value = float(value)
        print("Total value of the account: ", self.total_value)
    
    def accountSummaryEnd(self, reqId: int):
        print('end of account summary')
        value_event.set()

In [4]:
# Connect to the TWS API
app = App('127.0.0.1', 7497, 1000)
init_event.wait() # wait until it's connected
init_event.clear()

openOrder. orderId:17, contract:3691937,AMZN,STK,,0,?,,SMART,,USD,AMZN,NMS,False,,,,combo:, order:17,1000,860175866: MKT SELL 100@104 GTC
orderStatus. orderId: 17, status: PreSubmitted, filled: 0, remaining:100, avgFillPrice: 0.0, permId:860175866, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0


In [5]:
app.reqMarketDataType(1) # in case it doesn't work, change 1 to 3

ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm.nj


NextValidId: 18


ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:cashfarm


In [6]:
#get S&P tickers list from wikipedia
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []

for row in table.findAll('tr')[1:]:
    ticker_name = row.findAll('td')[0].text
    tickers.append(ticker_name)

#clean list
tickers = [s.replace('\n', '') for s in tickers]

#get data from yfin
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime.now().strftime("%Y-%m-%d")
data = yf.download(tickers, start=start, end=end)

print(data)
#TODO: note we might also need to get data from tws.

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted
                            Adj Close                                     \
                                    A        AAL         AAP        AAPL   
Date                                                                       
2010-01-04 00:00:00-05:00   20.301361   4.496877   37.163269    6.515211   
2010-01-05 00:00:00-05:00   20.080832   5.005957   36.942379    6.526477   
2010-01-06 00:00:00-05:00   20.009481   4.798553   37.264488    6.422666   
2010-01-07 00:00:00-05:00   19.983540   4.939965   37.255287    6.410791   
2010-01-08 00:00:00-05:00   19.977058   4.845691   37.402546    6.453412   
...                               ...        ...         ...         ...   
2023-01-25 00:00:00-05:00  155.759995  16.260000  149.110001  141.860001   
2023-01-26 00:00:00-05:00  15

In [7]:
data.to_pickle('SP500_data.pkl')

# data cleaning part

In [8]:
#clean data
df = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
df.set_index('Date', inplace=True)

#drop tickers not in the list
tickers.remove("BRK.B")
tickers.remove("BF.B")
tickers.remove("FTV")
tickers.remove("GEHC")

#sort by group
groups = df.groupby("Symbol")
Ticker_Data = {}

for t in tickers:
    Ticker_Data[t] = groups.get_group(t)

In [9]:
df

,Symbol,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2010-01-04 00:00:00-05:00,A,20.301361,22.389128,22.625179,22.267525,22.453505,3815561.0
2010-01-05 00:00:00-05:00,A,20.080832,22.145924,22.331903,22.002861,22.324751,4186031.0
2010-01-06 00:00:00-05:00,A,20.009481,22.067240,22.174536,22.002861,22.067240,3243779.0
2010-01-07 00:00:00-05:00,A,19.983540,22.038628,22.045780,21.816881,22.017166,3095172.0
2010-01-08 00:00:00-05:00,A,19.977058,22.031473,22.067240,21.745352,21.917025,3733918.0
...,...,...,...,...,...,...,...
2023-01-25 00:00:00-05:00,ZTS,165.509995,165.509995,166.720001,163.779999,165.440002,1804400.0
2023-01-26 00:00:00-05:00,ZTS,168.240005,168.240005,168.300003,166.119995,166.869995,1530700.0
2023-01-27 00:00:00-05:00,ZTS,165.179993,165.179993,167.690002,164.500000,166.979996,1831700.0


In [10]:
for t in tickers:
    Ticker_Data[t]['High Shifted']=Ticker_Data[t]['High'].shift(1)
    Ticker_Data[t]['Low Shifted'] = Ticker_Data[t]['Low'].shift(1)
    Ticker_Data[t]['Close Shifted'] = Ticker_Data[t]['Close'].shift(1)
    
    Ticker_Data[t]['Upper BBand'], Ticker_Data[t]['Middle BBand'],Ticker_Data[t]['Lower BBand']= ta.BBANDS(Ticker_Data[t]['Close Shifted'],
                                                                                                       timeperiod=20,)
    
    Ticker_Data[t]['RSI'] = ta.RSI(np.array(Ticker_Data[t]['Close Shifted']), timeperiod=14)

    Ticker_Data[t]['Macd'], Ticker_Data[t]['Macd Signal'],Ticker_Data[t]['Macd Hist'] = ta.MACD(Ticker_Data[t]['Close Shifted'], fastperiod=12, slowperiod=26, 
                                                               signalperiod=9)

    Ticker_Data[t]['Momentum'] = ta.MOM(Ticker_Data[t]['Close Shifted'],timeperiod=12)
    
    Ticker_Data[t]['Returns'] = np.log(Ticker_Data[t]['Open']/Ticker_Data[t]['Open'].shift(1))
    Ticker_Data[t].dropna(inplace =True)

/tmp/ipykernel_144641/96338543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ticker_Data[t]['High Shifted']=Ticker_Data[t]['High'].shift(1)
/tmp/ipykernel_144641/96338543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ticker_Data[t]['Low Shifted'] = Ticker_Data[t]['Low'].shift(1)
/tmp/ipykernel_144641/96338543.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [11]:
# if we are going to do a multfactor strategy
# max_abs_scaler = preprocessing.MaxAbsScaler()

# for i in tickers:
#     Ticker_Data[i].dropna(inplace=True)
#     X = np.array(Ticker_Data[i].drop(['Symbol','Returns', 'High Shifted', 'Low Shifted', 'Close Shifted'],1))
#     X = max_abs_scaler.fit_transform(X)
#     Y = np.array(Ticker_Data[i]['Returns'])

# experiment

In [12]:
# # Get the stock data for a company
# ticker = "AAPL"
# stock_info = yf.Ticker(ticker).info

# # Calculate ROA
# net_income = stock_info["incomeStatementHistory"][0]["netIncome"]
# total_assets = stock_info["balanceSheetHistory"][0]["totalAssets"]
# ROA = net_income / total_assets

# # Calculate ROE
# shareholder_equity = stock_info["balanceSheetHistory"][0]["totalShareholderEquity"]
# ROE = net_income / shareholder_equity

# # Print the results
# print(f"ROA for {ticker}: {ROA:.2f}")
# print(f"ROE for {ticker}: {ROE:.2f}")



In [13]:
# stock_info.keys()

# query the current portfolio


In [14]:
app.reqAccountUpdates(True, "DU6587691") #TODO: replace the DU222526 with our account ID
port_event.wait()
port_event.clear()

# time.sleep(10)
last_portfolio = app.last_portfolio

In [15]:
last_portfolio.iloc[range(0,3),:]

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,AAPL,0,145.029999,0.00,0.000000,0.00,659.31
1,ALB,355,286.010010,101533.55,286.469648,-163.17,0.00
2,AMZN,0,104.959999,0.00,0.000000,0.00,1032.74


In [16]:
app.reqAccountUpdates(False, "DU6587691")#TODO: replace the DU222526 with our account ID
app.reqAccountSummary(9001, "All", "NetLiquidation")
value_event.wait()
value_event.clear()
# time.sleep(3)
total_value = app.total_value
app.cancelAccountSummary(9001)

ERROR:ibapi.wrapper:ERROR -1 2100 API client has been unsubscribed from account data.


Total value of the account:  1001701.59
end of account summary


In [17]:
total_value

1001701.59

# decide the target portfolio
* we need to know the weight of the target_portfolio, as well as the market price of each stocks
* total_value - 1000 to save 1000 dollars in case something goes wrong

# main part
# strategy implementation
* decide what to buy & sell, as well as the quantity

In [18]:
# decide which stocks to buy
sp500_momentum = []
for t in tickers:
    sp500_momentum.append([t, Ticker_Data[t]['Momentum'][-1]])

sp500_momentum = pd.DataFrame(sp500_momentum, columns = ['tickers', 'Momentum'])
# decide the portfolio list
portfolio_list = sp500_momentum.nlargest(10, 'Momentum')['tickers'].to_list()

# get the market price
for i, t in enumerate(portfolio_list):
    contract = Contract()
    contract.symbol = t
    contract.secType = "STK"
    contract.currency = "USD"
    contract.exchange = "SMART"
    app.reqMktData(i, contract, "", False, False, [])
    mkt_event.wait()
    mkt_event.clear()
    app.cancelMktData(i)

mkt_df = pd.DataFrame(app.mkt_price, columns=['reqId', 'mkt_price'])
assert sum(mkt_df.isnull().any()) == 0, "didn't get all the market price"
assert len(mkt_df) == len(portfolio_list), "didn't get all the market price"


Tick Price. Ticker Id: 0 tickType: 1 Price: 2414.0
Tick Price. Ticker Id: 0 tickType: 2 Price: 2497.98
Tick Price. Ticker Id: 0 tickType: 4 Price: 2443.63
Tick Price. Ticker Id: 0 tickType: 6 Price: 2462.24
Tick Price. Ticker Id: 0 tickType: 7 Price: 2390.15
Tick Price. Ticker Id: 0 tickType: 9 Price: 2434.1
Tick Price. Ticker Id: 0 tickType: 14 Price: 2428.49
Tick Price. Ticker Id: 1 tickType: 1 Price: 1647.0
Tick Price. Ticker Id: 1 tickType: 2 Price: 1660.0
Tick Price. Ticker Id: 1 tickType: 4 Price: 1651.75
Tick Price. Ticker Id: 1 tickType: 6 Price: 1667.9
Tick Price. Ticker Id: 1 tickType: 7 Price: 1617.15
Tick Price. Ticker Id: 1 tickType: 9 Price: 1646.38
Tick Price. Ticker Id: 1 tickType: 14 Price: 1645.28
Tick Price. Ticker Id: 2 tickType: 1 Price: 5046.0
Tick Price. Ticker Id: 2 tickType: 2 Price: 5389.0
Tick Price. Ticker Id: 2 tickType: 4 Price: 5345.0
Tick Price. Ticker Id: 2 tickType: 6 Price: 5376.0
Tick Price. Ticker Id: 2 tickType: 7 Price: 5185.12
Tick Price. Ticker 

In [19]:
target_portfolio = pd.DataFrame(columns=['tickers', 'weight', 'mkt_price'])
target_portfolio['tickers'] = portfolio_list
target_portfolio['weight'] = np.repeat(1/len(portfolio_list), len(portfolio_list))
target_portfolio['mkt_price'] = mkt_df['mkt_price']

In [20]:
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value - 1000) / target_portfolio['mkt_price'])

In [21]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,AAPL,0,145.029999,0.00,0.000000,0.00,659.31
1,ALB,355,286.010010,101533.55,286.469648,-163.17,0.00
2,AMZN,0,104.959999,0.00,0.000000,0.00,1032.74


In [22]:
target_portfolio

,tickers,weight,mkt_price,quantity
0,BKNG,0.1,2434.10,41.0
1,CMG,0.1,1646.38,60.0
2,NVR,0.1,5270.00,18.0
3,NOW,0.1,455.13,219.0
4,URI,0.1,440.95,226.0
5,TSLA,0.1,173.22,577.0
6,MKTX,0.1,363.85,275.0
7,SIVB,0.1,302.44,330.0
8,ALB,0.1,281.45,355.0
9,POOL,0.1,385.61,259.0


# figure out the best order of our order. (To save commission fee)

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [27]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []
# Create a list to store the stocks that need to buy
stocks_to_buy = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['tickers']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = last_portfolio_stock['quantity'].values[0]
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append([row['ticker'], quantity_diff])

        if row['quantity'] > last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = row['quantity'] - last_portfolio_quantity 
            # Add the stock to the list of stocks to buy
            stocks_to_buy.append([row['ticker'], quantity_diff])

# we also need to sell the stocks that are in last_portfolio but not in target_portfolio
# Find the difference between the two portfolios
new_tickers_to_sell = set(last_portfolio['ticker']).difference(set(target_portfolio['tickers']))

for ticker in new_tickers_to_sell:
    # Find the quantity of the stock to sell from the last_portfolio
    quantity = last_portfolio.loc[last_portfolio['ticker'] == ticker, 'quantity'].values[0]
    # append to stocks to sell
    if quantity != 0:
        stocks_to_sell.append([ticker, quantity])

# we also need to buy the stocks that are in target_protfolio but not in last_portfolio
# Find the difference between the two portfolios
new_tickers_to_buy = set(target_portfolio['tickers']).difference(set(last_portfolio['ticker']))

for ticker in new_tickers_to_buy:
    # Find the quantity of the stock to buy from the target_portfolio
    quantity = target_portfolio.loc[target_portfolio['tickers'] == ticker, 'quantity'].values[0]
    # append to stocks to buy
    if quantity != 0:
        stocks_to_buy.append([ticker, quantity])

In [28]:
stocks_to_buy

[['POOL', 259.0],
 ['MKTX', 275.0],
 ['NVR', 18.0],
 ['CMG', 60.0],
 ['URI', 226.0],
 ['BKNG', 41.0],
 ['SIVB', 330.0],
 ['NOW', 219.0],
 ['TSLA', 577.0]]

In [29]:
stocks_to_sell

[]

# place order

In [32]:
app.reqIds(-1) # require a new id
id_event.wait()
id_event.clear()
orderid = app.nextValidOrderId
for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1

    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

for stock in stocks_to_buy:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "BUY"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1
    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

NextValidId: 24
openOrder. orderId:25, contract:39195389,POOL,STK,,0,?,,SMART,,USD,POOL,NMS,False,,,,combo:, order:25,1000,860175873: MKT BUY 259@417 GTC
orderStatus. orderId: 25, status: PreSubmitted, filled: 0, remaining:259, avgFillPrice: 0.0, permId:860175873, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:26, contract:31918430,MKTX,STK,,0,?,,SMART,,USD,MKTX,NMS,False,,,,combo:, order:26,1000,860175874: MKT BUY 275@400 GTC
orderStatus. orderId: 26, status: PreSubmitted, filled: 0, remaining:275, avgFillPrice: 0.0, permId:860175874, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:27, contract:755864,NVR,STK,,0,?,,SMART,,USD,NVR,NVR,False,,,,combo:, order:27,1000,860175875: MKT BUY 18@5389 GTC
orderStatus. orderId: 27, status: PreSubmitted, filled: 0, remaining:18, avgFillPrice: 0.0, permId:860175875, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderI

openOrder. orderId:33, contract:76792991,TSLA,STK,,0,?,,SMART,,USD,TSLA,NMS,False,,,,combo:, order:33,1000,860175881: MKT BUY 577@182.5 GTC
orderStatus. orderId: 33, status: PreSubmitted, filled: 0, remaining:577, avgFillPrice: 0.0, permId:860175881, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0


In [33]:
app.disconnect()